In [1]:
# !rm -f minsearch.py
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [5]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [6]:
from openai import OpenAI

client = OpenAI(
    base_url = 'http://localhost:11434/v1/',
    api_key='ollama',
)

In [15]:
llm('write that this is a test')

' Certainly! Below is a simple demonstration in HTML and JavaScript to display the text "This is a test".\n\n```html\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <title>Test Display</title>\n</head>\n<body>\n\n<!-- This is where we will display the test message -->\n<div id="testMessage"></div>\n\n<!-- Including a script to dynamically set the content of the div element -->\n<script>\n// Function to show the test text\nfunction displayTest() {\n    document.getElementById("testMessage").innerHTML = "This is a test";\n}\n\n// Call the function on page load or whenever necessary\ndocument.addEventListener(\'DOMContentLoaded\', (event) => {\n    displayTest(); // Displaying the test message as soon as the DOM content loads\n});\n</script>\n\n</body>\n</html>\n```\n\nThis simple example demonstrates how to create a web page that displays the phrase "This is a test" when loaded. The JavaScript code includes a function, `displayTest()`, which sets the text of a